In [1]:
import pyspark
try:
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')

## Ejercicio:
### El servicio meteorologico registra datos del tiempo para todas las ciudades donde posee una base de medicion.
### Esta informacion se encuentra en 2 RDD.
### En el primero se tiene informacion de las bases de medicion: (ID_BASE, NOMBRE, PCIA, CIUDAD, LAT, LON)
### El segundo posee informacion sobre las mediciones en si: (TIMESTAMP, ID_BASE, TEMPERATURA, HUMEDAD, PRESION, DIRECCION_VIENTO, VELOCIDAD_VIENTO).
### Se desea obtener un reporte para las bases de la Provincia de buenos Aires. El mismo debe informar los ID y nombre de bases de medicion que hayan registrado una variacion de temperatura promedio mensual mayor al 30% en el año 2018 (dada la temperatura con el promedio del mes anterior, para determinar la variacion porcentual).

# Creo datos iniciales que luego voy a paralelizar

In [60]:
#Id_base, nombre, pcia, ciudad, lat, lon
datos_bases = [(1,'basecita','bs.as','temp',10,10),
              (2,'base2','bs.as','lom',10,10),
              (3,'basecilla','bs.as','lan',10,10),
              (4,'LA BASE','bs.as','cons',10,10),
              (5,'basecita2','bs.as','ave',10,10),
              (6,'base51','salta','salta',10,10)]

#Timestamp, id_base, temperatura, humedad, presion, direccion_viento, velocidad_viento
datos_medic = [('2017-12-11',1,19.5,99,1,'n',1000),
               ('2017-12-11',2,19.5,99,1,'n',1000),
               ('2017-12-11',3,100,99,1,'n',1000),
               ('2017-12-11',4,19.5,99,1,'n',1000),
               ('2017-12-11',5,19.5,99,1,'n',1000),
               ('2016-05-11',2,19.5,99,1,'n',1000),
               ('2016-05-11',3,100,99,1,'n',1000),
               ('2016-05-11',4,19.5,99,1,'n',1000),
               ('2016-05-11',5,19.5,99,1,'n',1000),
               ('2016-05-11',1,19.5,99,1,'n',1000),
               ('2018-01-11',1,19.5,99,1,'n',1000),
               ('2018-01-11',2,21,99,1,'n',1000),
               ('2018-01-11',3,100,99,1,'n',1000),
               ('2018-02-11',3,100,99,1,'n',1000),
               ('2018-01-11',4,4,99,1,'n',1000),
               ('2018-01-11',5,12,99,1,'n',1000),
               ('2018-02-11',1,123,99,1,'n',1000),
               ('2018-02-11',2,13,99,1,'n',1000),
               ('2018-02-11',3,100,99,1,'n',1000),
               ('2018-02-11',4,5,99,1,'n',1000),
               ('2018-02-11',5,28,99,1,'n',1000),
               ('2018-02-11',1,9,99,1,'n',1000),
               ('2018-02-11',2,1,99,1,'n',1000),
               ('2018-02-11',3,100,99,1,'n',1000),
               ('2018-02-11',4,127,99,1,'n',1000),
               ('2018-02-11',5,14,99,1,'n',1000),
               ('2018-03-11',1,1000,99,1,'n',1000),
               ('2018-03-11',1,2,99,1,'n',1000),
               ('2018-03-11',2,21,99,1,'n',1000),
               ('2018-03-11',3,100,99,1,'n',1000),
               ('2018-03-11',4,25,99,1,'n',1000),
               ('2018-03-11',5,33,99,1,'n',1000),
               ('2018-03-11',2,12,99,1,'n',1000),
               ('2018-04-11',1,14,99,1,'n',1000),
               ('2018-04-11',2,13,99,1,'n',1000),
               ('2018-04-11',3,100,99,1,'n',1000),
               ('2018-04-11',4,12,99,1,'n',1000),
               ('2018-04-11',5,15,99,1,'n',1000),
               ('2018-05-11',1,19,99,1,'n',1000),
               ('2018-05-11',2,23,99,1,'n',1000),
               ('2018-05-11',3,100,99,1,'n',1000),
               ('2018-05-11',4,1,99,1,'n',1000),
               ('2018-05-11',5,33,99,1,'n',1000),
               ('2018-05-11',1,41,99,1,'n',1000),
               ('2018-05-11',2,2,99,1,'n',1000),
               ('2018-05-11',3,100,99,1,'n',1000),
               ('2018-05-11',4,44,99,1,'n',1000),
               ('2018-05-11',5,33,99,1,'n',1000),
               ('2018-06-11',1,1,99,1,'n',1000),
               ('2018-06-11',2,2,99,1,'n',1000),
               ('2018-06-11',3,100,99,1,'n',1000),
               ('2018-06-11',4,3,99,1,'n',1000),
               ('2018-06-11',5,1,99,1,'n',1000),
               ('2018-06-11',1,22,99,1,'n',1000),
               ('2018-06-11',2,33,99,1,'n',1000),
               ('2018-06-11',3,100,99,1,'n',1000),
               ('2018-06-11',4,22,99,1,'n',1000),
               ('2018-06-11',5,11,99,1,'n',1000),
               ('2018-07-11',1,11,99,1,'n',1000),
               ('2018-07-11',2,11,99,1,'n',1000),
               ('2018-07-11',3,100,99,1,'n',1000),
               ('2018-07-11',4,11,99,1,'n',1000),
               ('2018-07-11',5,11,99,1,'n',1000),
               ('2018-08-11',1,11,99,1,'n',1000),
               ('2018-08-11',2,11,99,1,'n',1000),
               ('2018-08-11',3,100,99,1,'n',1000),
               ('2018-08-11',4,11,99,1,'n',1000),
               ('2018-08-11',5,11,99,1,'n',1000),
               ('2018-09-11',1,11,99,1,'n',1000),
               ('2018-09-11',2,11,99,1,'n',1000),
               ('2018-09-11',3,100,99,1,'n',1000),
               ('2018-09-11',4,11,99,1,'n',1000),
               ('2018-09-11',5,11,99,1,'n',1000),
               ('2018-10-11',1,11,99,1,'n',1000),
               ('2018-10-11',2,11,99,1,'n',1000),
               ('2018-10-11',3,100,99,1,'n',1000),
               ('2018-10-11',4,11,99,1,'n',1000),
               ('2018-10-11',5,11,99,1,'n',1000),
               ('2018-11-11',1,11,99,1,'n',1000),
               ('2018-11-11',2,11,99,1,'n',1000),
               ('2018-11-11',3,100,99,1,'n',1000),
               ('2018-11-11',4,11,99,1,'n',1000),
               ('2018-11-11',5,11,99,1,'n',1000),
               ('2018-12-11',1,11,99,1,'n',1000),
               ('2018-12-11',2,11,99,1,'n',1000),
               ('2018-12-11',3,100,99,1,'n',1000),
               ('2018-12-11',4,11,99,1,'n',1000),
               ('2018-12-11',5,11,99,1,'n',1000)]
len(datos_medic)

88

In [51]:
rdd_med=sc.parallelize(datos_medic)
rdd_bas=sc.parallelize(datos_bases)

## Algunas pruebas antes de hacer la solucion

### Obtengo tuplas (Id base, lista mes y promedio temp) usando reduce by key

In [53]:
%%time
rdd_med.map(lambda x: ((x[1],x[0]),(x[2],1))).reduceByKey(lambda x,y: (x[0] + y[0],x[1] + y[1]))\
                    .map(lambda x: (x[0][0], [(x[0][1], x[1][0]/x[1][1])] ) )\
                    .reduceByKey(lambda x,y: x+y)\
                    .collect()

CPU times: user 45.3 ms, sys: 8.23 ms, total: 53.5 ms
Wall time: 610 ms


[(2,
  [('2016-05-11', 19.5),
   ('2018-04-11', 13.0),
   ('2018-05-11', 12.5),
   ('2018-06-11', 17.5),
   ('2018-07-11', 11.0),
   ('2018-10-11', 11.0),
   ('2018-11-11', 11.0),
   ('2018-12-11', 11.0),
   ('2017-12-11', 19.5),
   ('2018-01-11', 21.0),
   ('2018-02-11', 7.0),
   ('2018-03-11', 16.5),
   ('2018-08-11', 11.0),
   ('2018-09-11', 11.0)]),
 (4,
  [('2016-05-11', 19.5),
   ('2018-04-11', 12.0),
   ('2018-05-11', 22.5),
   ('2018-06-11', 12.5),
   ('2018-07-11', 11.0),
   ('2018-10-11', 11.0),
   ('2018-11-11', 11.0),
   ('2018-12-11', 11.0),
   ('2017-12-11', 19.5),
   ('2018-01-11', 4.0),
   ('2018-02-11', 66.0),
   ('2018-03-11', 25.0),
   ('2018-08-11', 11.0),
   ('2018-09-11', 11.0)]),
 (1,
  [('2017-12-11', 19.5),
   ('2018-01-11', 19.5),
   ('2018-02-11', 66.0),
   ('2018-03-11', 501.0),
   ('2018-08-11', 11.0),
   ('2018-09-11', 11.0),
   ('2016-05-11', 19.5),
   ('2018-04-11', 14.0),
   ('2018-05-11', 30.0),
   ('2018-06-11', 11.5),
   ('2018-07-11', 11.0),
   ('20

### Obtengo tuplas (Id base, lista mes y promedio temp) usando group by key

In [54]:
%%time
## Si no hago mapValues(list) en este caso devuelve un iterable (y si no hago el mapValues entonces tarda más
## cuando hago el collect, supongo que el tipo de dato iterable debe ser más pesado de levantar)
rdd_med.map(lambda x: ((x[1],x[0]),(x[2],1))).reduceByKey(lambda x,y: (x[0] + y[0],x[1] + y[1]))\
                    .map(lambda x: (x[0][0], (x[0][1], x[1][0]/x[1][1]) ) )\
                    .groupByKey().mapValues(list)\
                    .collect()

CPU times: user 60.8 ms, sys: 4.01 ms, total: 64.9 ms
Wall time: 665 ms


[(2,
  [('2016-05-11', 19.5),
   ('2018-04-11', 13.0),
   ('2018-05-11', 12.5),
   ('2018-06-11', 17.5),
   ('2018-07-11', 11.0),
   ('2018-10-11', 11.0),
   ('2018-11-11', 11.0),
   ('2018-12-11', 11.0),
   ('2017-12-11', 19.5),
   ('2018-01-11', 21.0),
   ('2018-02-11', 7.0),
   ('2018-03-11', 16.5),
   ('2018-08-11', 11.0),
   ('2018-09-11', 11.0)]),
 (4,
  [('2016-05-11', 19.5),
   ('2018-04-11', 12.0),
   ('2018-05-11', 22.5),
   ('2018-06-11', 12.5),
   ('2018-07-11', 11.0),
   ('2018-10-11', 11.0),
   ('2018-11-11', 11.0),
   ('2018-12-11', 11.0),
   ('2017-12-11', 19.5),
   ('2018-01-11', 4.0),
   ('2018-02-11', 66.0),
   ('2018-03-11', 25.0),
   ('2018-08-11', 11.0),
   ('2018-09-11', 11.0)]),
 (1,
  [('2017-12-11', 19.5),
   ('2018-01-11', 19.5),
   ('2018-02-11', 66.0),
   ('2018-03-11', 501.0),
   ('2018-08-11', 11.0),
   ('2018-09-11', 11.0),
   ('2016-05-11', 19.5),
   ('2018-04-11', 14.0),
   ('2018-05-11', 30.0),
   ('2018-06-11', 11.5),
   ('2018-07-11', 11.0),
   ('20

### Prueba que hice para el ordenamiento de lista de tuplas

In [55]:
def take_first(x):
    return x[0]

lista_tuplas = [(1,3),(3,5),(2,3)]

sorted(lista_tuplas, key = take_first)

[(1, 3), (2, 3), (3, 5)]

### Veo que se obtiene el modulo de un numero usando ABS

In [56]:
a=-1
abs(a)

1

## Recuerdo el ejercicio para evitar scrollear hasta arriba:
### El servicio meteorologico registra datos del tiempo para todas las ciudades donde posee una base de medicion.
### Esta informacion se encuentra en 2 RDD.
### En el primero se tiene informacion de las bases de medicion: (ID_BASE, NOMBRE, PCIA, CIUDAD, LAT, LON)
### El segundo posee informacion sobre las mediciones en si: (TIMESTAMP, ID_BASE, TEMPERATURA, HUMEDAD, PRESION, DIRECCION_VIENTO, VELOCIDAD_VIENTO).
### Se desea obtener un reporte para las bases de la Provincia de buenos Aires. El mismo debe informar los ID y nombre de bases de medicion que hayan registrado una variacion de temperatura promedio mensual mayor al 30% en el año 2018 (dada la temperatura con el promedio del mes anterior, para determinar la variacion porcentual).

# Solucion del ejercicio: (Actualizada)

### Correccion a la solucion

In [59]:
def obtener_fecha(x):
    fecha = x.split('-')
    return (fecha[0] + '-' + fecha[1])
    
def tomar_primer_elemento(x):
    return x[0]

def hubo_dif_30_porciento(x):
    lista_meses = sorted(list(x[1]), key=tomar_primer_elemento)
    hubo_dif = False
    for i in range(1,len(lista_meses)):
        if ( (abs( (lista_meses[i][1] - lista_meses[i-1][1]) / (lista_meses[i-1][1]) ) > 0.3) ) :
            hubo_dif = True
    return hubo_dif


rdd_bas_filt=rdd_bas.filter(lambda x: x[2] == 'bs.as').map(lambda x: (x[0],x[1]))


rdd_med.filter(lambda x: ( x[0].split('-')[0] == '2018' ) | ((x[0].split('-')[0] == '2017') & (x[0].split('-')[1] == '12')))\
                    .map(lambda x: (x[1],(obtener_fecha(x[0]),x[2])))\
                    .join(rdd_bas_filt)\
                    .map(lambda x: ( (x[0],x[1][1],x[1][0][0]), (x[1][0][1],1)) )\
                    .reduceByKey(lambda x,y: (x[0] + y[0],x[1] + y[1]))\
                    .map(lambda x: ((x[0][0], x[0][1]), (x[0][2], x[1][0]/x[1][1]) ) )\
                    .groupByKey()\
                    .filter(hubo_dif_30_porciento)\
                    .map(lambda x: (x[0][0],x[0][1]))\
                    .collect()

[(1, 'basecita'), (5, 'basecita2'), (2, 'base2'), (4, 'LA BASE')]

# Obtuve una lista de tuplas (id_base, nombre_base) con las bases que en alguno de los 12 meses registraron una diferencia superior al 30% con respecto al mes anterior en el promedio de temperatura (sin considerar enero)

In [58]:
rdd_med.map(lambda x: obtener_fecha(x[0])).take(5)

['2017-12', '2017-12', '2017-12', '2017-12', '2017-12']